# Applied ML: Homework Exercise 02-1

## Goal

After this exercise, you should be able to navigate the building blocks of Bayesian optimization (BO) using `scikit-optimize` for general black box optimization problems, and more specifically, hyperparameter optimization (HPO).

## Introduction

This section is a deep dive into Bayesian optimization (BO), also known as Model Based Optimization (MBO). BO is more complex than other tuning methods, so we will motivate theory and methodology first.

## Black Box Optimization

In hyperparameter optimization, learners are passed a hyperparameter configuration and evaluated on a given task via a resampling technique to estimate its generalization performance with the goal to find the optimal hyperparameter configuration. In general, this is a black box optimization problem, which considers the optimization of a function whose mathematical structure is unknown or unexploitable. The only thing we can observe is the generalization performance of the function given a hyperparameter configuration. As evaluating the performance of a learner can take a lot of time, HPO is an expensive black box optimization problem.

## Bayesian Optimization

There are many ways of doing black box optimization, grid and random search being examples of simple strategies. Bayesian optimization is a class of black box optimization algorithms that rely on a surrogate model trained on observed hyperparameter evaluations to model the black box function. This surrogate model tries to capture the unknown function between hyperparameter configurations and estimated generalization performance using (the very low number of) observed function evaluations. During each iteration, BO algorithms employ an acquisition function to determine the next candidate point for evaluation. This function measures the expected utility of each point within the search space based on the prediction of the surrogate model. The algorithm then selects the candidate point with the best acquisition function value and evaluates the black box function at that point to then update the surrogate model. This iterative process continues until a termination criterion is met, such as reaching a pre-specified maximum number of evaluations or achieving a desired level of performance. BO is a powerful method that often results in good optimization performance, especially if the cost of the black box evaluation becomes expensive and the optimization budget is tight.

In the rest of this section, we will first provide an introduction to black box optimization with the scikit-optimize package and then introduce the building blocks of BO algorithms and examine their interplay and interaction during the optimization process before we assemble these building blocks in a ready-to-use black box optimizer with `scikit-optimize`.

## Prerequisites

Let’s load the packages required for this exercise:

In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(111)

Before we apply BO to hyperparameter optimization (HPO), we try to optimize the following simple sinusoidal function:

In [2]:
from typing import Dict

# Define the sinusoidal function; expects a dict-like input with keys 'x1' and 'x2'
def sinus_1D(xs: Dict[str, float]) -> float:
    return 2 * xs['x1'] * np.sin(14 * xs['x1']) * np.sin(xs['x2']) * xs['x2']

## 1 Building Blocks of BO

Bayesian optimization (BO) usually follows this process:

1. Generate and evaluate an initial design

2. Loop:

    2.1. Fit a surrogate model on the archive of all observations made so far to model the unknown black box function.

    2.2. Optimize an acquisition function to determine which points of the search space are promising candidate(s) that should be evaluated next.

    2.3. Evaluate the next candidate(s) and update the archive of all observations made so far.

    2.4. Check if a given termination criterion is met; if not, go back to step 2.1.

The acquisition function relies on the mean and standard deviation prediction of the surrogate model and requires no evaluation of the true black box function, making it comparably cheap to optimize. A good acquisition function will balance exploiting knowledge about regions where we observed that performance is good and the surrogate model has low uncertainty with exploring regions where it has not yet evaluated points and as a result the uncertainty of the surrogate model is high.

BO is a highly modular algorithm: as long as the above structure is in place, then the surrogate models, acquisition functions, and acquisition function optimizers are all interchangeable to a certain extent. The design of `scikit-optimize` reflects this modularity, with key elements including surrogate models (e.g., Gaussian Process), acquisition functions (e.g., Expected Improvement), and optimizers for the acquisition functions. Let’s explore the interplay and interaction of these building blocks during optimization.

## 1.1 Initial design

The initial set of points evaluated before a surrogate model can be fit is referred to as the initial design. `scikit-optimize` allows you to either construct this manually or use built-in methods to automate this step. We will demonstrate a manual method here.

To construct an initial design, we will first try grid search, assuming an initial design of nine points on a domain of two numeric variables ranging from 0 to 1:

In [3]:
# Create a grid design for two variables ranging from 0 to 1 with resolution 3
x_vals = np.linspace(0, 1, 3)
# Use pd.DataFrame for the purpose of pretty print
grid_design = pd.DataFrame([(x1, x2) for x1 in x_vals for x2 in x_vals], columns=['x1', 'x2'])
print(f"Grid design:\n{grid_design}\n")

Grid design:
    x1   x2
0  0.0  0.0
1  0.0  0.5
2  0.0  1.0
3  0.5  0.0
4  0.5  0.5
5  0.5  1.0
6  1.0  0.0
7  1.0  0.5
8  1.0  1.0



As you can see, this is essentially a DataFrame encoding the set of hyperparameter configurations we want to evaluate first, before any of the real BO magic starts.

### Task

Construct a more refined initial design, using SciPy to implement a Sobol design with 30 points, which has better coverage properties than grid or random search. If you are interested in why the Sobol design has favorable properties, you can take a look at the original paper by [Niederreiter (1988)](https://www.sciencedirect.com/science/article/pii/0022314X8890025X?via%3Dihub).

<details><summary>Hint: </summary> 
    Wrap the code as a function as we will use it later. The signature should be `def get_sobol_design(n: int, d: int = 2) -> pd.DataFrame:` and the function uses [`scipy.stats.qmc.Sobol`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Sobol.html) sequence to generate `n` points in a `d`-dimensional unit cube.
</details>

In [4]:
#===SOLUTION===

from scipy.stats import qmc
import warnings


# Use SciPy's Sobol sequence to generate 30 points in a 2-dimensional unit cube
def get_sobol_design(n: int, d: int = 2) -> pd.DataFrame:
    sobol_sampler = qmc.Sobol(d=d, scramble=True)
    # Suppress the warning: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        sobol_design_array = sobol_sampler.random(n)
    sobol_design = pd.DataFrame(sobol_design_array, columns=[f'x{i}' for i in range(1, d+1)])

    return sobol_design

sobol_design = get_sobol_design(30)
print(f"First 5 rows of Sobol design:\n{sobol_design.head()}")

First 5 rows of Sobol design:
         x1        x2
0  0.833708  0.714247
1  0.393502  0.301624
2  0.166035  0.927012
3  0.606692  0.088859
4  0.641449  0.768583


## 1.2 Generate data from initial design

Unlike R version which manually creates an `ObjectiveRFun` instance, in Python we simply apply the sinus_1D function to the Sobol design, obtaining the target. We store the target as an additional column to `sobol_design` DataFrame, for the purpose of pretty print.

In [5]:
#===SOLUTION===

archive_data = get_sobol_design(30)
archive_data['y'] = archive_data.apply(lambda row: sinus_1D(row), axis=1)

archive_data.head(10)


,x1,x2,y
0,0.970342,0.645288,0.641158
1,0.160936,0.213977,0.011351
2,0.360321,0.977642,-0.552262
3,0.552327,0.412843,0.181627
4,0.731361,0.817742,-0.634655
5,0.415546,0.260273,-0.024989
6,0.122744,0.555186,0.071058
7,0.805533,0.116548,-0.020973
8,0.862980,0.905423,-0.572770
9,0.053580,0.454092,0.014551


## 1.3 Train surrogate model

A surrogate model wraps a regression learner that models the unknown black box function based on observed data. In `scikit-optimize`, surrogate models are typically implemented using regression learners such as Gaussian Processes.

Here, we'll use a Gaussian Process regressor with a Matérn kernel. The Matérn covariance function is a kernel used in Gaussian processes to model the smoothness of the random function, offering a flexible class of smoothness parameters. The BFGS algorithm is a type of quasi-Newton method used for optimization, particularly effective in maximizing the likelihood in Gaussian process models by efficiently finding parameter estimates.

Task: Update the surrogate model by fitting the Gaussian process and inspect the trained model's kernel parameters.

<details><summary>Hint 1:</summary>
    Use `GaussianProcessRegressor` and `sklearn.gaussian_process.kernels.Matern`.
</details>

In [6]:
#===SOLUTION===

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings(action='ignore', category=ConvergenceWarning, module='sklearn')

# Define a Gaussian Process regressor with a Matérn kernel (nu=2.5 corresponds to matern5_2)
kernel = Matern(nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernel, random_state=123)

# Prepare training data: features are x1 and x2; target is y from the archive_data DataFrame
X = archive_data[['x1', 'x2']]
y = archive_data['y']

# Update the surrogate model (i.e., fit the Gaussian process on the training data)
gpr.fit(X, y)

# Inspect the trained surrogate model's kernel (this shows the learned kernel parameters)
print(f"Trained surrogate model kernel: {gpr.kernel_}")

Trained surrogate model kernel: Matern(length_scale=1e-05, nu=2.5)


### 1.4 Implementation of the Acquisition Function

Implement the Expected Improvement (EI) acquisition function.

The function should have the following docstring:

```python
def make_neg_ei(y_best: np.ndarray, gaussian_process: GaussianProcessRegressor) -> Callable[[ndarray], float]:
    """
    Define a closure to generate a negative expected improvement function that captures the current y_best
    """
```

<details><summary>Hint 1:</summary>
    Use `skopt.aquisition.gaussian_ei` as the acquisition function.
</details>

<details><summary>Hint 2:</summary>
    You need to define a negative expeteced improvement function that captures the current `y_best`. Why negative ei? Because `x_next = argmax_x EI(x)`, but the `scipy.optimize` solves argmin problems.
</details>

In [7]:
#===SOLUTION===

from skopt.acquisition import gaussian_ei 


def make_neg_ei(y_best, gaussian_process):
    """
    Define a closure to generate a negative expected improvement function that captures the current y_best
    Why negative ei? Because x_next = argmax_x EI(x), but the scipy.optimize solves argmin problems.
    """

    def _neg_ei(x):
        x = x.reshape(1, -1)
        # Compute expected improvement using skopt's gaussian_ei; xi controls exploration-exploitation
        ei = gaussian_ei(x, gaussian_process, y_opt=y_best, xi=0.01)
        return -ei[0]
    return _neg_ei

## 2 Automating Bayesian Optimization
Note that due to the discrepancy between mlr3 and scikit-optimize, we dont't need Section 1.5 in the R exercise.

**Task**: Find the optimal hyperparameter configuration by iteratively updating the surrogate model and optimizing the acquisition function.

For convenience, you can utilize the `skopt.gp_minimize` function, a readily available Bayesian Optimization tool that internally handles the steps outlined in Exercise 1. This allows you to specify Bayesian Optimization details directly through the arguments of `skopt.gp_minimize`, eliminating the need to manually define an acquisition function.

In [8]:
#===SOLUTION===

from skopt import gp_minimize

# Create wrapper to adapt sinus_1D to the format expected by skopt
def objective(x):
    return sinus_1D({'x1': x[0], 'x2': x[1]})

# Define the search space
dimensions = [(0.0, 1.0), (0.0, 1.0)]

# Number of initial points and total evaluations
n_initial_points = 50
n_total_calls = 200

with warnings.catch_warnings():
    # ignore "UserWarning: The balance properties of Sobol' points require n to be a power of 2"
    warnings.simplefilter("ignore", UserWarning)
    # Run Bayesian optimization
    result = gp_minimize(
        func=objective,
        dimensions=dimensions,
        n_calls=n_total_calls - n_initial_points,
        n_initial_points=n_initial_points,
        initial_point_generator="sobol",
        acq_func="EI",
        random_state=111
    )

print(f"Best y in initial instance archive: {min(result.func_vals[:n_initial_points])}")
print(f"Initial evaluations: {n_initial_points}")

# Create dataframe with best result
best_x = result.x
best_y = result.fun
best_candidate = pd.DataFrame({'x1': [best_x[0]], 'x2': [best_x[1]], 'y': [best_y]})

print("\nFinal incumbent candidate from automated BO:")
print(best_candidate)

Best y in initial instance archive: -0.9516148029280513
Initial evaluations: 50

Final incumbent candidate from automated BO:
         x1        x2         y
0  0.791804  0.999493 -1.326099


## 3 BO for HPO with BayesSearch CV:

In this task, your goal is to apply Bayesian hyperparameter optimization using `BayesSearchCV` from `scikit-optimize` to tune the hyperparameters of an SVM classifier with an RBF kernel on the sonar dataset.

You should perform the following steps:

1. Load and prepare the sonar dataset from OpenML.

In [9]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml("sonar", version=1, as_frame=True, return_X_y=True)

2. Define the SVM classifier and specify a hyperparameter search space for the C (cost) and gamma parameters, using log-uniform distributions.

3. Define a scoring function to measure misclassification error.

5. Configure and run Bayesian optimization using 3-fold stratified cross-validation.

6. Identify and report the best hyperparameter configuration and its performance.

The search space is:

* `C`: `[1e-5, 1e5]`, log-scale.
  
* `gamma`

In [10]:
#===SOLUTION===

from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold

# Define the SVM classifier with a radial (RBF) kernel.
# In scikit-learn, the SVM's cost parameter is 'C'.
svm = SVC(kernel='rbf', probability=True)

# Define the hyperparameter search space:
# We use log-uniform distributions for 'C' (cost) and 'gamma'
search_spaces = {
    'C': (1e-5, 1e5, 'log-uniform'),
    'gamma': (1e-5, 1e5, 'log-uniform')
}

# Define a scorer for misclassification error.
def misclassification_error(y_true, y_pred):
    return np.mean(y_true != y_pred)

scorer = make_scorer(misclassification_error, greater_is_better=False)

# Setup 3-fold stratified cross-validation
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)

# Create the BayesSearchCV tuner.
tuner = BayesSearchCV(
    estimator=svm,
    search_spaces=search_spaces,
    n_iter=25,
    scoring=scorer,
    cv=cv,
    random_state=123
)

# Run the tuning procedure
tuner.fit(X, y)

# Extract the best hyperparameter configuration and performance.
best_hp = tuner.best_params_
# Note: the best_score_ is negative misclassification error.
best_misclassification_error = -tuner.best_score_

print("Best HP configuration (shown in original scale, not log-transformed)")
print(f"cost (C) = {best_hp['C']:.5f}, gamma = {best_hp['gamma']:.5f}")
print(f"Misclassification error: {best_misclassification_error:.6f}")

Best HP configuration (shown in original scale, not log-transformed)
cost (C) = 3856.87501, gamma = 0.18238
Misclassification error: 0.134714


## 4 Bonus: Implementing BO Loop from Scratch

In Exercise 2, you performed Bayesian Optimization using `skopt.gp_minimize`.

Alternatively, you can implement the Bayesian Optimization Loop from scratch. The following hints may be helpful.

<details><summary>Hint 1:</summary>
    Use `scipy.optimize.differential_evolution` as the optimizer for the acquisition function.
</details>


<details><summary>Hint 2:</summary>
The pseudo code is like this:
    
    1. Re-create the data (50 initial instances) as the initial archive data (which we will gradually expand).
    
    2. Define bounds corresponding to our domain.
    
    3. Define a closure to generate a negative expected improvement function.
    
    4. Set up how many instances we have, and how many instances (200) we are allowed to evaluate in total?

    5. Then we enter the loop, while termination condition is not met:

    6. Fit (update) the surrogate model using current archive data.

    7. Compute the best observed objective value (for minimization, the smallest y).

    8. Create a negative EI function based on the current best value.

    9. Optimize the acquisition function using differential evolution.

    10. Append the new candidate to the archive data.


In the end, we print out the best sample in the archive data.
</details>

In [11]:
#===SOLUTION===

from scipy.optimize import differential_evolution

# Re-create the data
archive_data = get_sobol_design(50)
archive_data['y'] = archive_data.apply(lambda row: sinus_1D(row), axis=1)
print(f"Best y in initial instance archive: {archive_data['y'].min()}")

# Define bounds corresponding to our domain
bounds = [(0, 1), (0, 1)]

n_evals = 200
current_evals = len(archive_data)
print(f"Initial evaluations: {current_evals}")
# In a typical MBO loop, we would update the surrogate and add new evaluations until termination.
while current_evals < n_evals:
    # Fit (update) the surrogate model using current archive data
    X = archive_data[['x1', 'x2']]
    y = archive_data['y']
    gpr.fit(X, y)
    
    # Compute the best observed objective value (for minimization, the smallest y)
    y_best = y.min()
    
    # Create a negative EI function based on the current best value
    neg_ei = make_neg_ei(y_best, gpr)
    
    # Optimize the acquisition function using differential evolution
    result = differential_evolution(neg_ei, bounds, maxiter=300, seed=123)
    candidate_x = result.x
    candidate_y = sinus_1D({'x1': candidate_x[0], 'x2': candidate_x[1]})
    
    # Append the new candidate to the archive
    new_row = {'x1': candidate_x[0], 'x2': candidate_x[1], 'y': candidate_y}
    archive_data = pd.concat([archive_data, pd.DataFrame([new_row])], ignore_index=True)
    current_evals = len(archive_data)

# At this point, the termination criterion is met.
# Retrieve the best candidate from the instance (the incumbent)
best_idx = archive_data['y'].idxmin()
best_candidate = archive_data.loc[[best_idx], ['x1', 'x2','y']]

print("\nFinal incumbent candidate from automated BO:")
print(best_candidate)

Best y in initial instance archive: -0.8605392053911493
Initial evaluations: 50

Final incumbent candidate from automated BO:
          x1        x2         y
75  0.837354  0.983681 -1.024403


## Summary

In this exercise, you learned how Bayesian Optimization can be effectively used in Python to solve general black-box optimization and hyperparameter optimization (HPO) problems specifically. Rather than evaluating arbitrary hyperparameter configurations, you optimized an acquisition function derived from a surrogate model (Gaussian Process) to iteratively propose new candidate configurations, efficiently identifying optimal hyperparameters.